<a href="https://colab.research.google.com/github/ItsShi/Mono-Depth/blob/main/visualise_pose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://drive.google.com/file/d/1SoD421qKt_4A5OsyQxykHniQlDpTuCi_/view?usp=sharing
import gdown

url = 'https://drive.google.com/uc?id=1SoD421qKt_4A5OsyQxykHniQlDpTuCi_'
gdown.download(url,'SCARED_videos.zip',quiet=True)

!unzip -q SCARED_videos.zip

# https://drive.google.com/file/d/1kf7LjQ6a2ACKr6nX5Uyee3of3bXn1xWB/view
url = 'https://drive.google.com/uc?id=1kf7LjQ6a2ACKr6nX5Uyee3of3bXn1xWB'
gdown.download(url,'sfmLearner.zip',quiet=True)
!unzip -q sfmLearner.zip

In [ ]:
!pip install ffmpeg

video_name = "d3k4"
!ffmpeg -i /content/videos/{video_name}_rgb.mp4 -filter:v "crop=1280:1024:0:0" /content/videos/{video_name}_crop_rgb.mp4

!mkdir -p /content/frames/{video_name}
!ffmpeg -i /content/videos/{video_name}_crop_rgb.mp4 %6d.jpg
!mv *.jpg /content/frames/{video_name}

import PIL.Image as pil
im = pil.open("/content/frames/{}/000001.jpg".format(video_name))
print(im.size)

In [ ]:
import gdown
import shutil
camera_url = 'https://drive.google.com/uc?id=1_G5h2CUXqdREPj8dB7DorVDQHxP4oLvF'
gdown.download(camera_url,'SCARED_camera.zip',quiet=True)
!unzip -q /content/SCARED_camera.zip
shutil.unpack_archive("SCARED_camera/{}_frame_data.tar.gz".format(video_name), "{}_json".format(video_name))

import gdown
import numpy as np
gt_poses_url = 'https://drive.google.com/uc?id=1s6_7XgmH3Lsam2wJhTFNQCNwsjJMptA6'
gdown.download(gt_poses_url,'gt_poses.npy',quiet=True)
poses = np.load("gt_poses.npy", allow_pickle=True).item()

with open("{}_sequence.txt".format(video_name), "w+") as f:
    for i in range(1, len(poses[video_name])):
        f.write("dataset{}/keyframe{}\t{}\tl\n".format(video_name[1], video_name[-1],i))


In [ ]:
def replace_text_in_file(search_path, replace_path, search_text, replace_text):
    with open(search_path, 'r') as file: content = file.read()
    for i in range(len(search_text)):
      modified_content = content.replace(search_text[i], replace_text[i])
      content = modified_content
    with open(replace_path, 'w') as file: file.write(modified_content)

In [ ]:
!pip -q install timm
!git clone https://github.com/ShuweiShao/AF-SfMLearner.git
!git clone https://github.com/noahzn/Lite-Mono.git
!git clone https://github.com/ItsShi/MonoDepth.git
!mkdir Lite-Mono/splits/endovis/
script_folder = "AF-SfMLearner"  # "AF-SfMLearner" #"Lite-Mono"
!cp AF-SfMLearner/evaluate_pose.py {script_folder}/evaluate_pose.py
!cp AF-SfMLearner/export_gt_pose.py {script_folder}/export_gt_pose.py
!cp AF-SfMLearner/visualize_pose.py {script_folder}/visualize_pose.py
!cp AF-SfMLearner/networks/pose_decoder.py {script_folder}/networks/pose_decoder.py
!cp AF-SfMLearner/networks/resnet_encoder.py {script_folder}/networks/resnet_encoder.py

!cp {video_name}_sequence.txt {script_folder}/splits/endovis/
!cp -r MonoDepth/datasets/ {script_folder}/

In [ ]:
search_text = ["odom_9", "load(pose_encoder_path)", "load(pose_decoder_path)", ".cuda()", "load(encoder_path)", "load(decoder_path)",
               "test_files_sequence2.txt",
               """gt_local_poses = np.load(gt_path, fix_imports=True, encoding='latin1')["data"]""",
               ]
replace_text = ["endovis", "load(pose_encoder_path, map_location='cpu')", "load(pose_decoder_path, map_location='cpu')",".cpu()", "load(encoder_path, map_location='cpu')", "load(decoder_path, map_location='cpu')",
                "{}_sequence.txt".format(video_name),
                """our_path = os.path.join(os.path.dirname(__file__), "splits", "endovis", "pose_our.npz")
    np.savez_compressed(our_path, data=np.array(pred_poses))
    return""",
                ]
replace_text_in_file("{}/evaluate_pose.py".format(script_folder), "{}/evaluate_pose.py".format(script_folder), search_text, replace_text)
replace_text_in_file("{}/test_simple.py".format(script_folder), "{}/test_simple.py".format(script_folder), search_text, replace_text)

search_text = ["          folder,",
               '  "image_02/data/frame_data"',
              #  "gt_poses.npz",
               '"test_files.txt"',
               ]
replace_text = ["          folder[0] + folder[7] + folder[9] + folder[-1] + '_json',",
                '  # "image_02/data/frame_data"',
                # "gt_poses_sq2.npz",
                '"{}_sequence.txt"'.format(video_name)
                ]
replace_text_in_file("{}/export_gt_pose.py".format(script_folder), "{}/export_gt_pose.py".format(script_folder), search_text, replace_text)

replace_text_in_file("{}/visualize_pose.py".format(script_folder), "{}/visualize_pose.py".format(script_folder), ['"curve",', 'gca'], ['', 'add_subplot'])

if script_folder == "Lite-Mono":
  search_path, replace_path = "{}/options.py".format(script_folder), "{}/options.py".format(script_folder)
  replace_text_in_file(search_path, replace_path, ['''choices=["eigen_zhou"''', '''choices=["kitti"''', "eigen"], ['''choices=["eigen_zhou", "endovis"''', '''choices=["kitti", "endovis"''', "endovis"])

  search_text = ["MonodepthOptions",
                ]
  replace_text = ["LiteMonoOptions",
                  ]
  replace_text_in_file("{}/evaluate_pose.py".format(script_folder), "{}/evaluate_pose.py".format(script_folder), search_text, replace_text)

  search_text = ['''elif height == 320 and width == 1024:'''
                ]

  replace_text = ['''elif (height == 320 and width == 1024) or (height == 256 and width == 320):'''
                ]
  search_path, replace_path = "{}/networks/depth_encoder.py".format(script_folder), "{}/networks/depth_encoder.py".format(script_folder)
  replace_text_in_file(search_path, replace_path, search_text, replace_text)

In [ ]:
!python {script_folder}/export_gt_pose.py --split endovis --data_path /content/
!python {script_folder}/evaluate_pose.py --data_path frames/{video_name} --eval_split endovis --load_weights_folder Model_trained_end_to_end
!python {script_folder}/visualize_pose.py
from google.colab import files
files.download('vo.png')

from PIL import Image
Image.show('vo.png')


In [ ]:
!python {script_folder}/export_gt_pose.py --split endovis --data_path /content/
!python {script_folder}/evaluate_pose.py --data_path frames/{video_name} --eval_split endovis --load_weights_folder Model_trained_end_to_end
!python {script_folder}/visualize_pose.py
from google.colab import files
files.download('vo.png')

from PIL import Image
Image.show('vo.png')